In [12]:
import requests
ngrams_api = "https://api.ngrams.dev"
corpus_id = "eng"
data = {"query":"chimp", "flags": "cs"}
r = requests.get(f"{ngrams_api}/{corpus_id}/search", params=data)
r.json()

{'queryTokens': [{'text': 'chimp', 'type': 'TERM'}],
 'ngrams': [{'id': '1b10fd2d586ef5c52a322f66c0f5179a',
   'absTotalMatchCount': 251006,
   'relTotalMatchCount': 1.2565909557086897e-07,
   'tokens': [{'text': 'chimp', 'type': 'TERM'}]}]}

In [23]:
import numpy as np
word_pairs = [("chimp", "chimpanzee"), ("info", "information"), ("dorm", "dormitory")]
word_data = {}
for wp in word_pairs:
    for idx, word in enumerate(wp):
        data = {"query": word, "flags": "cs"}
        request_id = requests.get(f"{ngrams_api}/{corpus_id}/search", params=data)
        id = request_id.json()["ngrams"][0]["id"]
        ngram_request = requests.get(f"{ngrams_api}/{corpus_id}/{id}", params=data)
        ngram_request = ngram_request.json()
        word_data[word] = {"start_year":ngram_request["stats"][0]["year"]}
        word_data[word]["start_rel_count"] = ngram_request["stats"][0]["relMatchCount"]
        word_data[word]["end_year"] = ngram_request["stats"][-1]["year"]
        word_data[word]["final_rel_count"] = ngram_request["stats"][-1]["relMatchCount"]
        points = [ngram_request["stats"][i]["relMatchCount"] for i in range(len(ngram_request["stats"]))]
        max_slope = max([x - z for x, z in zip(points[:-1], points[1:])])
        max_slope_id = np.argmax([x - z for x, z in zip(points[:-1], points[1:])])
        word_data[word]["max_slope"] = max_slope
        word_data[word]["max_slope_year"] = ngram_request["stats"][max_slope_id]["year"]
        # TODO find maybe decade with biggest increase
for wp in word_pairs: 
    if word_data[wp[0]]["final_rel_count"]> word_data[wp[1]]["final_rel_count"]:
        print(wp, "shorter word more popular now")
    else:
        print(wp, "longer word more popular now")
word_data


('chimp', 'chimpanzee') longer word more popular now
('info', 'information') longer word more popular now
('dorm', 'dormitory') shorter word more popular now


{'chimp': {'start_year': 1706,
  'start_rel_count': 7.728846706904589e-08,
  'end_year': 2019,
  'final_rel_count': 2.025308493964661e-07,
  'max_slope': 5.3373676134432925e-08,
  'max_slope_year': 1942},
 'chimpanzee': {'start_year': 1523,
  'start_rel_count': 2.0853796120776845e-06,
  'end_year': 2019,
  'final_rel_count': 6.629676279292064e-07,
  'max_slope': 2.0622884925804945e-06,
  'max_slope_year': 1523},
 'info': {'start_year': 1501,
  'start_rel_count': 3.2394508094253764e-07,
  'end_year': 2019,
  'final_rel_count': 2.9698391262354393e-06,
  'max_slope': 7.53557623768214e-05,
  'max_slope_year': 1554},
 'information': {'start_year': 1472,
  'start_rel_count': 0.0005524767959745691,
  'end_year': 2019,
  'final_rel_count': 0.0002847724370683589,
  'max_slope': 0.0014490783769568402,
  'max_slope_year': 1506},
 'dorm': {'start_year': 1616,
  'start_rel_count': 1.118673342909177e-06,
  'end_year': 2019,
  'final_rel_count': 2.1492890917998327e-06,
  'max_slope': 9.57689554981696